# PTH to ONNX

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from torch.autograd import Variable
import torch.onnx
import torchvision
import torch
import torchvision.models as models

In [ ]:
#dummy_input = Variable(torch.randn(1, 3, 256, 256))

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/pytorch/MY_MODEL.pth')

In [ ]:
model = models.alexnet(pretrained=True)


In [ ]:
dummy_input = torch.randn(10, 3, 224, 224)

In [ ]:
torch.onnx.export(model, dummy_input, "MY_MODEL.onnx", opset_version = 11)

In [ ]:
!cp /content/MY_MODEL.onnx /content/drive/MyDrive/

# Convert ONNX to TFLITE

In [ ]:
!pip install onnx

In [ ]:
import onnx
from collections import OrderedDict
import tensorflow as tf
from torch.autograd import Variable

In [ ]:
!pip install onnx_tf

In [ ]:
from onnx_tf.backend import prepare

In [ ]:
model = onnx.load("/content/MY_MODEL.onnx")


In [ ]:
tf_rep = prepare(model)

In [ ]:
print('inputs:', tf_rep.inputs)
print('outputs:', tf_rep.outputs)
print('tensor_dict:')
print(tf_rep.tensor_dict)

In [ ]:
tf_rep.export_graph("%s/mnist.pb")

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("%s/mnist.pb")

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tf_lite_model = converter.convert()

In [ ]:
open('MY_MODEL.tflite', 'wb').write(tf_lite_model)

In [ ]:
!cp /content/MY_MODEL.tflite /content/drive/MyDrive/